In [3]:
from datasets import load_dataset
ds = load_dataset("osunlp/MagicBrush", split="train")
print(ds.features)
print(len(ds))
print(ds[0])


/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating dev split: 100%|██████████| 528/528 [00:02<00:00, 235.60 examples/s]

{'img_id': Value('string'), 'turn_index': Value('int32'), 'source_img': Image(mode=None, decode=True), 'mask_img': Image(mode=None, decode=True), 'instruction': Value('string'), 'target_img': Image(mode=None, decode=True)}
8807
{'img_id': '327726', 'turn_index': 1, 'source_img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=500x500 at 0x7B3E259A6FE0>, 'mask_img': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1024x1024 at 0x7B3E25F728F0>, 'instruction': 'change the table for a dog', 'target_img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7B3E25F71E40>}


In [11]:
# === IMPORTS ===
import os
import random
import torch
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

# === REPRODUCIBILITY ===
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# === SETTINGS ===
base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_save_path = "./adapters/slm_lora_magicbrush"
csv_path = "data/instruction_image_pairs_magicbrush.csv"
image_dir = "data/magicbrush_images"
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 2
batch_size = 4
lr = 3e-5
max_length = 128
image_size = 512

# === DATASET CLASS ===
class InstImgDataset(Dataset):
    def __init__(self, csv_path, image_dir, tokenizer, max_length=128, image_size=512):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.tokenizer = tokenizer
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor()
        ])
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        instr = row["instruction"]
        img_path = os.path.join(self.image_dir, row["image_path"])
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            # fallback: blank image if fail
            image = Image.new("RGB", (image_size, image_size), color=(255,255,255))
        image = self.transform(image)
        tokens = self.tokenizer(instr, return_tensors="pt",
                                truncation=True, max_length=max_length).input_ids.squeeze(0)
        return {"instruction_ids": tokens, "image": image, "instruction_text": instr}

# === LOAD MODELS ===
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16 if device!="cpu" else torch.float32
).to(device)

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1
)
model = get_peft_model(base_model, config)

# === DATA LOADER ===
dataset = InstImgDataset(csv_path, image_dir, tokenizer, max_length=max_length, image_size=image_size)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# === OPTIMIZER ===
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
model.train()

# === TRAINING LOOP ===
for epoch in range(epochs):
    for batch in loader:
        instr_ids = batch["instruction_ids"].to(device)
        outputs = model(input_ids=instr_ids, labels=instr_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1}/{epochs} — Loss: {loss.item():.4f}")

# === SAVE ADAPTER ===
model.save_pretrained(adapter_save_path)
print(f"✅ Adapter saved at: {adapter_save_path}")

# === INTEGRATION TEST SNIPPET ===
from models.slm_model import load_slm
from models.diffusion_model import load_diffusion_model

tokenizer2, slm2 = load_slm(base_model_name, adapter_path=adapter_save_path)
diffusion = load_diffusion_model("runwayml/stable-diffusion-v1-5")

test_prompt = "A futuristic castle on a floating island under golden clouds"
inputs = tokenizer2(test_prompt, return_tensors="pt").to(device)
outputs = slm2.generate(**inputs, max_new_tokens=40)
refined = tokenizer2.decode(outputs[0], skip_special_tokens=True)

print("Refined prompt:", refined)
image = diffusion(refined).images[0]
save_path = "results/integration_test_refined.png"
image.save(save_path)
print(f"✅ Generated image saved at {save_path}")


Error importing huggingface_hub._snapshot_download: cannot import name 'HF_HUB_ENABLE_HF_TRANSFER' from 'huggingface_hub.constants' (/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/constants.py)


ImportError: cannot import name 'HF_HUB_ENABLE_HF_TRANSFER' from 'huggingface_hub.constants' (/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/constants.py)

In [3]:
!pip uninstall -y transformers diffusers huggingface_hub peft accelerate


In [4]:
import sys
print(sys.executable)


/home/cse-sdpl/anaconda3/envs/gpu_env/bin/python


In [5]:
!pip install diffusers==0.26.3
!pip install huggingface_hub==0.20.3
!pip install transformers==4.37.2
!pip install peft==0.10.0
!pip install accelerate==0.25.0
!pip install datasets


  Using cached diffusers-0.26.3-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-1.1.4-py3-none-any.whl.metadata (13 kB)
Using cached diffusers-0.26.3-py3-none-any.whl (1.9 MB)
Using cached huggingface_hub-1.1.4-py3-none-any.whl (515 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [diffusers]/2 [diffusers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
tokenizers 0.15.2 requires huggingface_hub<1.0,>=0.16.4, but you have huggingface-hub 1.1.4 which is incompatible.
  Using cached huggingface_hub-0.20.3-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.1.4
    Uninstalling huggingface_hub-1.1.4:
      Successfully un

In [6]:
import diffusers, huggingface_hub, transformers

print("Diffusers:", diffusers.__version__)
print("HF Hub:", huggingface_hub.__version__)
print("Transformers:", transformers.__version__)


/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/__init__.py)

In [7]:
!pip uninstall -y huggingface_hub


Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0


In [8]:
!pip uninstall -y huggingface-hub


In [9]:
!rm -rf /home/cse-sdpl/.local/lib/python3.10/site-packages/huggingface_hub*


In [10]:
!rm -rf /home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub*


In [11]:
!pip install huggingface_hub==0.20.3 --no-cache-dir


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.4.1 requires huggingface-hub<2.0,>=0.25.0, but you have huggingface-hub 0.20.3 which is incompatible.
gradio 5.49.1 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.20.3 which is incompatible.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.


In [12]:
import huggingface_hub
print(huggingface_hub.__version__)


0.36.0


In [13]:
import huggingface_hub
print(huggingface_hub.__file__)
print(huggingface_hub.__version__)


/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/__init__.py
0.36.0


In [14]:
import shutil, glob

paths = glob.glob("/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/__init__.py", recursive=True)
for p in paths:
    print("Deleting:", p)
    shutil.rmtree(p, ignore_errors=True)

print("Done.")


Deleting: /home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/__init__.py
Done.


In [17]:
import shutil
shutil.rmtree("/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub", ignore_errors=True)
shutil.rmtree("/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub-0.36.0.dist-info", ignore_errors=True)
print("Deleted old huggingface_hub.")


Deleted old huggingface_hub.


In [21]:
import glob
glob.glob("/home/cse-sdpl/.local/lib/python3.10/site-packages/huggingface_hub*")


[]

In [22]:
import huggingface_hub
print(huggingface_hub.__file__)
print(huggingface_hub.__version__)


/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/__init__.py
0.36.0


In [23]:
!pip uninstall -y huggingface_hub diffusers transformers datasets gradio peft accelerate pillow


Found existing installation: huggingface-hub 0.20.3
Uninstalling huggingface-hub-0.20.3:
  Successfully uninstalled huggingface-hub-0.20.3
Found existing installation: diffusers 0.26.3
Uninstalling diffusers-0.26.3:
  Successfully uninstalled diffusers-0.26.3
Found existing installation: transformers 4.37.2
Uninstalling transformers-4.37.2:
  Successfully uninstalled transformers-4.37.2
Found existing installation: datasets 4.4.1
Uninstalling datasets-4.4.1:
  Successfully uninstalled datasets-4.4.1
Found existing installation: gradio 5.49.1
Uninstalling gradio-5.49.1:
  Successfully uninstalled gradio-5.49.1
Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
Found existing installation: pillow 12.0.0
Uninstalling pillow-12.0.0:
  Successfully uninstalled pillow-12.0.0


In [24]:
import shutil, glob

paths = glob.glob("/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub*", recursive=True)
for p in paths:
    print("Deleting:", p)
    shutil.rmtree(p, ignore_errors=True)

print("Done.")


Done.


In [25]:
for pkg in ["transformers", "diffusers", "datasets", "gradio", "peft"]:
    paths = glob.glob(f"/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/{pkg}*", recursive=True)
    for p in paths:
        print("Deleting:", p)
        shutil.rmtree(p, ignore_errors=True)


Deleting: /home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/gradio_client
Deleting: /home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/gradio_client-1.13.3.dist-info


In [26]:
glob.glob("/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/*huggingface*")


[]

In [27]:
!pip install huggingface_hub==0.20.3 --no-cache-dir
!pip install transformers==4.37.2 --no-cache-dir
!pip install diffusers==0.26.3 --no-cache-dir
!pip install accelerate==0.25.0 --no-cache-dir
!pip install peft==0.10.0 --no-cache-dir
!pip install datasets==2.19.0 --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [diffusers]/2 [diffusers]
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 19.9 MB/s  0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.18
    Uninstalling multiprocess-0.70.18:
      Successfully uninstalled multipro

In [28]:
import huggingface_hub
print(huggingface_hub.__version__)
print(huggingface_hub.__file__)


0.36.0
/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/__init__.py


In [29]:
print(huggingface_hub.__version__)


0.36.0


In [30]:
!pip show huggingface_hub


Name: huggingface_hub
Version: 1.1.4
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages
Requires: filelock, fsspec, hf-xet, httpx, packaging, pyyaml, shellingham, tqdm, typer-slim, typing-extensions
Required-by: accelerate, datasets, diffusers, peft, tokenizers, transformers


In [31]:
print(huggingface_hub.__file__)


/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/__init__.py
